In [1]:
## Fast Import
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys, os
import string
import pathlib
import time
import math, random
import pprint
import yaml
from collections import OrderedDict
from tqdm import tqdm

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import albumentations as A

np.set_printoptions(precision=3)
curr_path = pathlib.Path(os.getcwd())

cards = !echo $SGE_HGR_gpu_card
device = torch.device(f"cuda:{cards[0]}" if torch.cuda.is_available() else 'cpu')
print(device)

sys.path.append(str(curr_path.parent / 'src'))
import lib
from lib.data.decathlon import dataset as decathlon

cuda:1


In [34]:
from lib.assess import metrics3d as M
from lib.assess import losses3d as L

pred = torch.randint(0, 2, (3, 2, 1, 2, 3))
targ = torch.randint(0, 2, (3, 1, 1, 2, 3))
d = L.cross_entropy_loss(pred, targ)
print(pred)
print(targ)
pprint.pprint(d)

tensor([[[[[1, 1, 0],
           [0, 0, 0]]],


         [[[1, 0, 0],
           [1, 0, 1]]]],



        [[[[1, 0, 1],
           [0, 1, 1]]],


         [[[1, 0, 1],
           [0, 0, 0]]]],



        [[[[0, 0, 1],
           [0, 1, 1]]],


         [[[1, 0, 0],
           [1, 1, 0]]]]])
tensor([[[[[0, 0, 1],
           [0, 0, 0]]]],



        [[[[0, 0, 0],
           [0, 0, 1]]]],



        [[[[1, 1, 0],
           [0, 0, 1]]]]])
{'dice_c': tensor([0.6364, 0.1538]),
 'fn': tensor([[3., 1.],
        [2., 1.],
        [1., 2.]]),
 'fn_c': tensor([6., 4.]),
 'fp': tensor([[0., 3.],
        [1., 2.],
        [1., 2.]]),
 'fp_c': tensor([2., 7.]),
 'hausdorff_c': None,
 'jaccard_c': tensor([0.4667, 0.0833]),
 'sensitivity_c': tensor([0.5385, 0.2000]),
 'tp': tensor([[2., 0.],
        [3., 0.],
        [2., 1.]]),
 'tp_c': tensor([7., 1.])}


### Helper Functions

In [43]:
def split_default_df(df, train=0.6, val=0.2, test=0.2):
    df = df.copy(deep=True)
    N = len(df)
    N_train = math.ceil(N * train)
    if N_train % 2 == 1:
        N_train += 1
    N_test = math.ceil(N * test)
    N_val = N - N_train - N_test
    
    indices = set(range(N))
    train_indices = set(random.sample(indices, k=N_train))
    indices = indices.difference(train_indices)
    test_indices = set(random.sample(indices, k=N_test))
    val_indices = indices.difference(test_indices)
    
    for i, S in df.iterrows():
        subset = 'train'
        if i in test_indices:
            subset = 'test'
        elif i in val_indices:
            subset = 'val'
        S['subset'] = subset
    print(f"Out of {N} exs, sampled {len(train_indices)}-{len(val_indices)}-{len(test_indices)}")
    return df
    

# Decathlon

In [4]:
df_d = decathlon.get_dfs()

In [45]:
### Loop through dataframes and ..
#    1. save default csv

for i, task in enumerate(decathlon.TASKS):
    if not task: continue
    print(f'Task {i+1}: {task}')
    tdf = df_d[task]
    img_path = pathlib.Path(tdf.iloc[0]['image'])
    task_path = img_path.parent.parent
    # tdf.to_csv(task_path / 'default_df.csv')
    
    splits_path = task_path / 'splits'
    if not splits_path.exists():
        splits_path.mkdir()
    lab_df = tdf[tdf['subset'] == 'train']
    
    for version in ('a', 'b', 'c'):
        split_df = split_default_df(lab_df)
        split_df.to_csv(splits_path / f'{task.lower()}_60-20-20split_{version}.csv')

Task 3: Liver
Out of 131 exs, sampled 80-24-27
Out of 131 exs, sampled 80-24-27
Out of 131 exs, sampled 80-24-27
Task 6: Lung
Out of 63 exs, sampled 38-12-13
Out of 63 exs, sampled 38-12-13
Out of 63 exs, sampled 38-12-13
Task 7: Pancreas
Out of 281 exs, sampled 170-54-57
Out of 281 exs, sampled 170-54-57
Out of 281 exs, sampled 170-54-57
Task 8: HepaticVessel
Out of 303 exs, sampled 182-60-61
Out of 303 exs, sampled 182-60-61
Out of 303 exs, sampled 182-60-61
Task 9: Spleen
Out of 41 exs, sampled 26-6-9
Out of 41 exs, sampled 26-6-9
Out of 41 exs, sampled 26-6-9
Task 10: Colon
Out of 126 exs, sampled 76-24-26
Out of 126 exs, sampled 76-24-26
Out of 126 exs, sampled 76-24-26
